# 2 Computational Statistics

## ベイズの定理

$$ p(H|D) = \dfrac{p(H)p(D|H)}{p(D)} $$

いつものライブラリ

In [108]:
import pandas as pd
import seaborn as sns
sns.set_style("whitegrid")
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
%matplotlib inline

# 2.1 分布

In [168]:
class Pmf():
    
    """Probability Mass Function（確率質量関数）"""
    
    def __init__(self):
        self.hypos = []
        self.priors = {}
        self.priors_given_likelihoods = {}
        self.posteriors = {}
    
    def Set(self, x, prob):
        self.hypos.append(x)
        self.priors[x] = prob
    
    def Values(self):
        return self.hypos

    def Incr(self, x, i):
        self.priors[x] = self.priors[x] + 1
        
    def Mult(self, x, likelihood):
        self.priors_given_likelihoods[x] = self.priors[x] * likelihood
        
    def Normalize(self):
        for x in self.priors_given_likelihoods.keys():
            self.posteriors[x] = self.priors_given_likelihoods[x] / (sum(self.priors_given_likelihoods.values()))

    def Items(self):
        return self.posteriors.items()
            
    def Prob(self, x):
        return self.posteriors[x]

# 2.2 クッキー問題

In [169]:
pmf = Pmf()
pmf.Set("Bowl 1", 0.5)
pmf.Set("Bowl 2", 0.5)
print(pmf.priors)

pmf.Mult("Bowl 1", 0.75)
pmf.Mult("Bowl 2", 0.5)

pmf.Normalize()

print(pmf.posteriors)

print(pmf.Prob("Bowl 1"))

{'Bowl 1': 0.5, 'Bowl 2': 0.5}
{'Bowl 1': 0.6, 'Bowl 2': 0.4}
0.6


# 2.3 ベイジアンフレームワーク

In [170]:
class Cookie(Pmf):
    
    def __init__(self, hypos):
        Pmf.__init__(self) # TODO ここの書き方がよくわからない。Mix-inみたいなの？
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
        
    def Likelihood(self, data, hypo):
        
        mixes = {
            "Bowl 1": {"vanilla": 0.75, "chocolate": 0.25},
            "Bowl 2": {"vanilla": 0.5, "chocolate": 0.5}
        }
        
        self.mixes = mixes
        
        mix = self.mixes[hypo]
        like = mix[data]
        return like
        
    def Update(self, data):
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()

In [171]:
hypos = ["Bowl 1", "Bowl 2"]
pmf = Cookie(hypos)

In [172]:
pmf.Update("vanilla")

In [175]:
for hypo, prob in pmf.Items():
    print("{}: {}".format(hypo, prob))

Bowl 1: 0.6
Bowl 2: 0.4
